In [ ]:
data = {
    "KNN": [],
    "SVM": [],
    "DRSN": [],
    "DT": [],
    "RF": [],
    "LR": [],
    "ANN": []
}
df = pd.DataFrame(data)
columns = df.columns.drop('DRSN')  
for col in columns:
    t_stat, p_value = stats.ttest_rel(df['DRSN'], df[col])
    print(f"t  - {col} vs DRSN：")
    print(f"t : {t_stat:.4f}, p : {p_value:.20f}")
    if p_value < 0.05:
        print("（p < 0.05）\n")
    else:
        print("（p ≥ 0.05）\n")

In [ ]:
for col in df.columns:

    data_col = df[col]
    mean_val = data_col.mean()
    std_val = data_col.std()
    n = len(data_col)
    t_crit = stats.t.ppf(0.975, df=n-1)
    se = std_val / np.sqrt(n)
    lower = mean_val - t_crit * se
    upper = mean_val + t_crit * se
    
    print(f"=== {col}  95% CI ===")
    print(f"m: {mean_val:.4f}")
    print(f"sd: {std_val:.4f}")
    print(f"95% ci: ({lower:.4f}, {upper:.4f})\n")

In [ ]:
def cohen_d(x, y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    s_pooled = np.sqrt(((nx-1) * np.std(x, ddof=1)**2 + (ny-1) * np.std(y, ddof=1)**2) / dof)
    return (np.mean(x) - np.mean(y)) / s_pooled

drsn_algorithm = "DRSN"
other_algorithms = [col for col in df.columns if col != drsn_algorithm]

results = pd.DataFrame(columns=['Compared Algorithm', 'Mean Difference', 'Cohen\'s d', 'Effect Size'])

for alg in other_algorithms:
    mean_diff = df[drsn_algorithm].mean() - df[alg].mean()
    d = cohen_d(df[drsn_algorithm], df[alg])
    
    if abs(d) < 0.2:
        effect_size = 'Very Small'
    elif abs(d) < 0.5:
        effect_size = 'Small'
    elif abs(d) < 0.8:
        effect_size = 'Medium'
    else:
        effect_size = 'Large'
    
    results.loc[len(results)] = [alg, mean_diff, d, effect_size]
results = results.sort_values(by='Cohen\'s d', key=abs, ascending=False)
print("DRSN vs (Cohen's d):")
print("="*60)
print("{:<15} {:<15} {:<15} {:<15}".format("Compared Algorithm", "Mean Difference", "Cohen's d", "Effect Size"))
print("-"*60)
for _, row in results.iterrows():
    print("{:<15} {:<15.4f} {:<15.4f} {:<15}".format(
        row['Compared Algorithm'], 
        row['Mean Difference'], 
        row['Cohen\'s d'], 
        row['Effect Size']
    ))

In [ ]:
data_5pct = []

data_10pct = []

data_20pct = []

statistic, p_value = stats.kruskal(data_5pct, data_10pct, data_20pct)

print("Kruskal-Wallis:")
print(f"H = {statistic:.4f}")
print(f"p = {p_value:.6f}")


if p_value < 0.05:
    print(f" (p={p_value:.4f} < 0.05)")
else:
    print(f"(p={p_value:.4f} ≥ 0.05)")

In [ ]:
import libpysal as lps
import matplotlib.pyplot as plt
from math import inf

data = pd.read_csv('.csv') 
points = np.column_stack((data['longitude'], data['latitude']))
values = data['ndvi'].values

w_distance = lps.weights.DistanceBand.from_coordinates(
    points, threshold=0.5, binary=True)


w_knearest = lps.weights.KNN.from_coordinates(points, k=4)


w_inverse = lps.weights.Distance.from_coordinates(
    points, threshold=1.0, binary=False, transform='inverse'
)


moran = lps.esda.moran.Moran(values, w_distance)


print(f"I: {moran.I:.4f}")
print(f"Z: {moran.z_score:.2f}")
print(f"P: {moran.p:.4f}")

alpha = 0.05 
is_significant = (abs(moran.z_score) > 1.96) and (moran.p < alpha)
if is_significant:
    if moran.I > 0:
        print(f"p<{alpha}")
    else:
        print(f"p<{alpha}")
else:
    print(f"p≥{alpha}")